## Transform Payments Data
1. Extract Date and Time from payment_timestamp and create new columns payment_date and payment_time
1. Map payment_status to contain descriptive values  
   (1-Success, 2-Pending, 3-Cancelled, 4-Failed)
1. Write transformed data to the Silver schema 

In [0]:
df = spark.read.table("gizmobox.bronze.payments")
display(df) 

In [0]:
# 1.
from pyspark.sql import functions as F

df_extracted_payments =  (
                    df
                    .select(
                        'payment_id',
                        'order_id',
                        F.date_format('payment_timestamp', 'yyyy-MM-dd').cast('date').alias('payment_date'),
                        F.date_format('payment_timestamp', 'HH:mm:ss').alias('payment_time'),
                        'payment_status',
                        'payment_method'
                    )   
)

display(df_extracted_payments)

In [0]:
# 2. 

df_mapped_payments = (
    df_extracted_payments
    .select(
        'payment_id',
        'order_id',
        'payment_date',
        'payment_time',
        F.when(df_extracted_payments.payment_status == 1, 'Success')
         .when(df_extracted_payments.payment_status == 2, 'Pending')
         .when(df_extracted_payments.payment_status == 3, 'Cancelled')
         .when(df_extracted_payments.payment_status == 4, 'Failed')
         .alias('payment_status'),
         'payment_method'
    )
)

display(df_mapped_payments)

In [0]:
df_mapped_payments.writeTo('gizmobox.silver.py_payments').createOrReplace()